In [1]:
%load_ext nb_black
# редактирование ячеек по стандартам pep8

<IPython.core.display.Javascript object>

In [2]:
reviews = []
names = []
studios = []
release_dates = []
genre_tags = []
rates = []
authors = []
dates = []  # даты написания отзыва
links = []  # ссылки на следующие страницы с отзывами на этот же фильм
links_csv = []

<IPython.core.display.Javascript object>

## Функция для обработки страниц с отзывами

In [3]:
def getting_reviews(soup_page, review_link):
    page_text = soup_page.find_all("div", {"class": "review pad_top1"})
    for i in page_text:
        try:
            reviews.append(
                i.find("span", {"class": "blurb blurb_expanded"}).text.strip(
                    "\n"
                )  # если есть полная версия отзыва, скачиваем её
            )
        except:
            reviews.append(i.find("div", {"class": "review_body"}).text.strip("\n"))
        try:  # добавление оценок
            rates.append(
                i.find(
                    "div", {"class": "metascore_w user large movie negative indiv"}
                ).text
            )
        except:
            try:
                rates.append(
                    i.find(
                        "div", {"class": "metascore_w user large movie mixed indiv"}
                    ).text
                )
            except:
                try:  # у положительных отзывов есть отдельный тег для максимальных оценок
                    rates.append(
                        i.find(
                            "div",
                            {"class": "metascore_w user large movie positive indiv"},
                        ).text
                    )
                except:
                    rates.append(
                        i.find(
                            "div",
                            {
                                "class": "metascore_w user large movie positive indiv perfect"
                            },
                        ).text
                    )
        authors.append(i.find("span", {"class": "author"}).text)
        dates.append(i.find("span", {"class": "date"}).text)
        links_csv.append(review_link)
        names.append(name)
        studios.append(studio)
        release_dates.append(release_date)
        genre_tags.append(genres)

<IPython.core.display.Javascript object>

## Запрос

In [4]:
import requests
import time
import random
from fake_useragent import UserAgent
from bs4 import BeautifulSoup

urls = [
    "https://www.metacritic.com/movie/cars-2",
    "https://www.metacritic.com/movie/inside-out-2015",
]

session = requests.session()

ua = UserAgent(verify_ssl=False)
headers = {"User-Agent": ua.random}
random.uniform(1, 3)

for url in urls:
    for _ in range(5):
        response = session.get(
            url,
            headers=headers,
        )
        time.sleep(random.uniform(1.1, 5.2))

    page = response.text
    soup = BeautifulSoup(page, "html.parser")
    # сбор метаинформации
    name = soup.find("div", {"class": "product_page_title oswald"}).text.split("\n")[1]
    studio = soup.find("span", {"class": "distributor"}).text.strip(" |")
    release_date = soup.find("span", {"class": "release_date"}).text.split("\n")[2]
    genres = (
        soup.find("div", {"class": "genres"})
        .text.split("\n")[3]
        .replace(" ", "")
        .split(",")
    )

    page_with_reviews = url + "/user-reviews"

    for _ in range(5):
        response = session.get(
            page_with_reviews,  # переход на страницу с отзывами
            headers=headers,
        )
        time.sleep(random.uniform(1.1, 5.2))

    page = response.text
    soup = BeautifulSoup(page, "html.parser")
    page_links = soup.find_all("a", {"class": "page_num"})[
        1:
    ]  # находим все страницы с отзывами

    for i in range(len(page_links)):
        links.append("https://www.metacritic.com" + page_links[i].get("href"))

    getting_reviews(soup, page_with_reviews)  # применяем функцию на текущей странице

for link in links:
    for _ in range(5):
        response = session.get(
            link,
            headers=headers,
        )
        time.sleep(random.uniform(1.1, 5.2))

    page = response.text
    soup = BeautifulSoup(page, "html.parser")
    getting_reviews(soup, link)

<IPython.core.display.Javascript object>

## Запись в csv-файл

In [5]:
import pandas as pd

<IPython.core.display.Javascript object>

In [22]:
data = {
    "review": reviews,
    "movie": names,
    "studio": studios,
    "release_date": release_dates,
    "genres": genre_tags,
    "rate": rates,
    "author": authors,
    "date": dates,
    "link": links_csv,
}

df = pd.DataFrame(data)
df = df[:150] # немного сокращаем df, чтобы корпус точно работал не медленно
df

,review,movie,studio,release_date,genres,rate,author,date,link
0,Lets start of with the good things. The action...,Cars 2,Walt Disney Studios Motion Pictures,"June 24, 2011","[Comedy, Animation]",5,AwesomeReviewer,"Mar 26, 2012",https://www.metacritic.com/movie/cars-2/user-r...
1,This was a heck of a lot better than what peop...,Cars 2,Walt Disney Studios Motion Pictures,"June 24, 2011","[Comedy, Animation]",7,Untitled94,"Aug 4, 2017",https://www.metacritic.com/movie/cars-2/user-r...
2,"My 3.5 year old boy loves all Pixar movies, as...",Cars 2,Walt Disney Studios Motion Pictures,"June 24, 2011","[Comedy, Animation]",10,BEER,"Aug 22, 2011",https://www.metacritic.com/movie/cars-2/user-r...
3,A good exciting cool car action movie with gad...,Cars 2,Walt Disney Studios Motion Pictures,"June 24, 2011","[Comedy, Animation]",8,MovieCritic17,"Aug 27, 2011",https://www.metacritic.com/movie/cars-2/user-r...
4,I read the reviews here before I decided on go...,Cars 2,Walt Disney Studios Motion Pictures,"June 24, 2011","[Comedy, Animation]",9,Vulcan,"Aug 31, 2011",https://www.metacritic.com/movie/cars-2/user-r...
...,...,...,...,...,...,...,...,...,...
145,The Best Pixar film ever. It might not be for ...,Inside Out,Walt Disney Studios Motion Pictures,"June 19, 2015","[Adventure, Drama, Fantasy, Comedy, Animation,...",10,LivingTribunal,"Jun 18, 2018",https://www.metacritic.com/movie/inside-out-20...
146,Do you wanna learn about your mind in a more e...,Inside Out,Walt Disney Studios Motion Pictures,"June 19, 2015","[Adventure, Drama, Fantasy, Comedy, Animation,...",10,danipp,"Jun 20, 2015",https://www.metacritic.com/movie/inside-out-20...
147,"If you like animated movies, do yourself a fav...",Inside Out,Walt Disney Studios Motion Pictures,"June 19, 2015","[Adventure, Drama, Fantasy, Comedy, Animation,...",10,Dufflad,"Jun 27, 2015",https://www.metacritic.com/movie/inside-out-20...
148,"With a smart, funny, sad, visually stunning an...",Inside Out,Walt Disney Studios Motion Pictures,"June 19, 2015","[Adventure, Drama, Fantasy, Comedy, Animation,...",10,AED,"Jun 20, 2015",https://www.metacritic.com/movie/inside-out-20...


<IPython.core.display.Javascript object>

In [23]:
df.to_csv("corpus.csv", index=None)

<IPython.core.display.Javascript object>